In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

**ANÁLISE EXPLORATÓRIA DO DATASET**

In [ ]:
df.head()

In [ ]:
df_test.head()

In [ ]:
df.shape

In [ ]:
df_test.shape

In [ ]:
df.dtypes

In [ ]:
df_test.dtypes

In [ ]:
df = df.drop('id', axis=1)
test_id = df_test['id']
df_test = df_test.drop('id', axis=1)

In [ ]:
df.head()

In [ ]:
df_test.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df_test.nunique()

In [ ]:
duplicatas = df_test.duplicated()
num_duplicatas = duplicatas.sum()
print("Total de duplicatas:", num_duplicatas)

In [ ]:
duplicatas = df.duplicated()
num_duplicatas = duplicatas.sum()
print("Total de duplicatas:", num_duplicatas)

In [ ]:
cmap = plt.cm.get_cmap('tab20', df['NObeyesdad'].nunique())

ax = df['NObeyesdad'].value_counts().sort_index()\
    .plot(kind='bar',
          title='Distribuição dos níveis de Obesidade',
          figsize=(7, 3),
          color=[cmap(i) for i in range(df['NObeyesdad'].nunique())])

ax.set_xlabel('NObeyesdad')
plt.show()

In [ ]:
count_nobeyesdad = df['NObeyesdad'].value_counts().sort_index()
print(count_nobeyesdad)


In [ ]:
colunas_categoricas = df.select_dtypes(include = ['object'])
colunas_categoricas

In [ ]:
colunas_categoricas_test = df_test.select_dtypes(include = ['object'])
colunas_categoricas_test

In [ ]:
df['NObeyesdad'].unique()

In [ ]:
dictionary = {'Insufficient_Weight': '0',
              'Normal_Weight': '1',
              'Overweight_Level_I' : '2',
              'Overweight_Level_II': '3',
              'Obesity_Type_I': '4',
              'Obesity_Type_II': '5',
              'Obesity_Type_III': '6'        
    
}

In [ ]:
df['NObeyesdad'] = df['NObeyesdad'].replace(dictionary).astype('int64')
df['NObeyesdad'].unique()

In [ ]:
df['CAEC'].unique()

In [ ]:
df_test['CAEC'].unique()

In [ ]:
df['CALC'].unique()

In [ ]:
df_test['CALC'].unique()

In [ ]:
caecdictionary = {'no': '0',
                  'Sometimes': '1',
                  'Frequently': '2',
                  'Always': '3'   
}

In [ ]:
df['CAEC'] = df['CAEC'].replace(caecdictionary).astype(int)
df['CAEC'].unique()

In [ ]:
df_test['CAEC'] = df_test['CAEC'].replace(caecdictionary).astype(int)
df_test['CAEC'].unique()

In [ ]:
calcdictionary = {'no': '0',
                  'Sometimes': '1',
                  'Frequently': '2',
                  'Always': '3'   
}

In [ ]:
df['CALC'] = df['CALC'].replace(calcdictionary).astype(int)
df['CALC'].unique()

In [ ]:
df_test['CALC'] = df_test['CALC'].replace(calcdictionary).astype(int)
df_test['CALC'].unique()

In [ ]:
df_test['MTRANS'].unique()

In [ ]:
df['MTRANS'].unique()

In [ ]:
mtransdictionary = {'Bike': '0',
              'Walking': '1',
              'Motorbike': '2',
              'Automobile': '3',
              'Public_Transportation': '4'          
}

In [ ]:
df['MTRANS'] = df['MTRANS'].replace(mtransdictionary).astype('int64')
df['MTRANS'].unique()

In [ ]:
df_test['MTRANS'] = df_test['MTRANS'].replace(mtransdictionary).astype('int64')
df_test['MTRANS'].unique()

In [ ]:
colunas_categoricas_test = df_test.select_dtypes(include = ['object'])
colunas_categoricas_test.columns

In [ ]:
colunas_categoricas = df.select_dtypes(include = ['object'])
colunas_categoricas.columns

In [ ]:
df = pd.get_dummies(df)
df_test = pd.get_dummies(df_test)

In [ ]:
df.columns

In [ ]:
df_test.columns

In [ ]:
df = df.drop(columns=['Gender_Male', 
                      'family_history_with_overweight_no', 
                      'FAVC_no', 
                      'SMOKE_no',
                      'SCC_no'                       
])

In [ ]:
df.columns

In [ ]:
df_test = df_test.drop(columns=['Gender_Male', 
                      'family_history_with_overweight_no', 
                      'FAVC_no', 
                      'SMOKE_no',
                      'SCC_no'                       
])

In [ ]:
df_test.columns

In [ ]:
df.info()

In [ ]:
df_test.info()

In [ ]:
df['IMC'] = ((df['Weight'])**0.5)/(df['Height']**1)
df['IMC'].value_counts()

In [ ]:
df_test['IMC'] = ((df_test['Weight'])**0.5)/(df_test['Height']**1)
df_test['IMC'].value_counts()

**MATRIZ DE CONFUSÃO**

In [ ]:
df = df.astype(float)
df_test = df_test.astype(float)

conf_matriz = df.corr()

sorted_corr = conf_matriz['NObeyesdad'].sort_index(ascending=True)
corr_columns = sorted_corr.index

subset_conf_matriz = conf_matriz.loc[corr_columns, corr_columns]

plt.figure(figsize=(12,12))
sns.heatmap(subset_conf_matriz, annot=True, cmap='Blues', linewidths=0.5, fmt=".2f", square=True)

plt.title("Matriz de Confusão")
plt.show()

In [ ]:
x = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

**CRIANDO O MODELO**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=3361)

In [ ]:
from xgboost import XGBClassifier
XGB = XGBClassifier(random_state=3361)
XGB.fit(x_train, y_train)
y_pred = XGB.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print(classification_report(y_pred, y_val))
print("A acurácia do modelo foi de {} %".format(round(100*accuracy_score(y_pred, y_val),2)))

In [ ]:
labels = ['Abaixo_do_peso', 
          'Peso_Normal', 
          'Sobrepeso_Nivel_I', 
          'Sobrepeso_Nivel_II', 
          'Obesidade_Tipo_I', 
          'Obesidade_Tipo_II', 
          'Obesidade_Tipo_III' ]
sns.heatmap(pd.DataFrame(confusion_matrix(y_pred, y_val)),annot=True, fmt= 'd', annot_kws = {'size': 10},
linewidths= 0.5, cmap = 'Oranges', cbar=False, xticklabels= labels, yticklabels=labels);
plt.title('Matriz de Confusão')
plt.ylabel('Verdadeiro')
plt.xlabel('Predição')
plt.show()

In [ ]:
cm = confusion_matrix(y_val, y_pred)
cm_df = pd.DataFrame(cm, 
                     index=['Abaixo_do_peso', 
                            'Peso_Normal', 
                            'Sobrepeso_Nivel_I', 
                            'Sobrepeso_Nivel_II', 
                            'Obesidade_Tipo_I', 
                            'Obesidade_Tipo_II', 
                            'Obesidade_Tipo_III'],
                     columns=['Abaixo_do_peso', 
                              'Peso_Normal', 
                              'Sobrepeso_Nivel_I', 
                              'Sobrepeso_Nivel_II', 
                              'Obesidade_Tipo_I', 
                              'Obesidade_Tipo_II', 
                              'Obesidade_Tipo_III'])

# Cálculo de acertos por classe do modelo em percentual
acuracia_por_classe = np.diag(cm_df) / np.sum(cm_df, axis=1)
acuracia_por_classe = pd.Series(acuracia_por_classe, index=cm_df.index)


acuracia_por_classe = acuracia_por_classe * 100
print(acuracia_por_classe)


In [ ]:
from sklearn.model_selection import cross_val_score

XGB_scores = cross_val_score(XGB, x_train, y_train, cv=5)
XGB_scores

In [ ]:
def mostrar_resultados(scores):
    print("Score:", scores.round(3))
    print("Média:", round(scores.mean(),3))
    print("Desvio padrão:", round(scores.std(),3))

mostrar_resultados(XGB_scores)

In [ ]:
import joblib

In [ ]:
joblib.dump(XGB, 'XGB.joblib')

In [ ]:
modelo_carregado = joblib.load('XGB.joblib')

In [ ]:
XGB_y_pred = modelo_carregado.predict(df_test)
XGB_y_pred

In [ ]:
modelo_resultados = pd.DataFrame({'id': test_id, 'NObeyesdad': XGB_y_pred})

In [ ]:
dictionary = {0: 'Abaixo do Peso',
              1: 'Peso_Normal',
              2: 'Sobrepeso_Nível_I',
              3: 'Sobrepeso_Nível_II',
              4: 'Obesidade_Tipo_I',
              5: 'Obesidade_Tipo_II',
              6: 'Obesidade_Tipo_III'
             }
modelo_resultados['NObeyesdad'] = modelo_resultados['NObeyesdad'].replace(dictionary)

In [ ]:
modelo_resultados.head(7)

In [ ]:
cmap = plt.cm.get_cmap('tab20', df['NObeyesdad'].nunique())

ax = modelo_resultados['NObeyesdad'].value_counts().sort_index()\
    .plot(kind='bar',
          title='Distribuição dos níveis de Obesidade',
          figsize=(7, 3),
          color=[cmap(i) for i in range(df['NObeyesdad'].nunique())])

ax.set_xlabel('NObeyesdad')
plt.show()

In [ ]:
count_nobeyesdad = modelo_resultados['NObeyesdad'].value_counts().sort_index()
print(count_nobeyesdad)


In [ ]:
modelo_resultados[['id','NObeyesdad']].to_csv('submission.csv', index=False)